In [1]:
import torch
import torch.optim
import numpy as np

from nni.nas.space import model_context
from nni.nas.strategy import DARTS as DartsStrategy
from nni.nas.experiment import NasExperiment
from dataset.classification import fetch_data
from evaluators.classification import ClassificationEvaluator
from models.mlp import MLP

In [2]:
np.random.seed(0)
torch.random.manual_seed(0)

# Fetch dataset loaders

In [3]:
task_config, loaders = fetch_data(batch_size=1024, num_workers=4)

In [4]:
for split_name, loader in loaders.items():
    print(split_name, 'dataset size:', len(loader.dataset))

print('num_classes:', task_config.out_features)

train dataset size: 371847
val dataset size: 92962
test dataset size: 116203
num_classes: 7


# Training

In [6]:
evaluator = ClassificationEvaluator(
    learning_rate=3e-4,
    weight_decay=1e-5,
    optimizer=torch.optim.AdamW,
    train_dataloaders=loaders['train'],
    val_dataloaders=loaders['val'],
    num_classes=task_config.out_features,
    max_epochs=200,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
strategy = DartsStrategy()
model_space = MLP(d_in=task_config.in_features, d_out=task_config.out_features, dropout=0.1)
experiment = NasExperiment(model_space, evaluator, strategy)
experiment.run()

[2024-05-14 12:44:40] Config is not provided. Will try to infer.
[2024-05-14 12:44:40] Strategy is found to be a one-shot strategy. Setting execution engine to "sequential" and format to "raw".


[2024-05-14 12:44:40] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 12:44:40] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 12:44:40] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 12:44:40] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 12:44:40] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 12:44:40] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 12:44:40] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 12:44:40] WARNING: `training_service` will be ignored for sequential execution engine.
[2024-05-14 12:44:40] WARNING: Checkpoint callback does not have last_model_path or best_model_path attribute. Either the strategy has not started, or it did not save any checkpoint: <pytorch_lightning.callba

You are using a CUDA device ('NVIDIA GeForce RTX 4060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                 | Params
---------------------------------------------------------
0 | training_module | ClassificationModule | 2.7 M 
---------------------------------------------------------
2.7 M     Trainable params
0         Non-trainable params
2.7 M     Total params
10.633    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


[2024-05-14 13:32:20] Waiting for models submitted to engine to finish...
[2024-05-14 13:32:20] Experiment is completed.
[2024-05-14 13:32:20] WARNING: `training_service` will be ignored for sequential execution engine.


True

# Train final model

In [8]:
exported_arch = experiment.export_top_models(formatter='dict')[0]
print(exported_arch)

{'MLP/d_block': 32, 'MLP/in_act': 0, 'MLP/n_blocks': 3, 'MLP/blocks_act': 0}


In [9]:
with model_context(exported_arch):
    final_model = MLP(d_in=task_config.in_features, d_out=task_config.out_features, dropout=0.1)

In [10]:
evaluator = ClassificationEvaluator(
    learning_rate=3e-4,
    weight_decay=1e-5,
    optimizer=torch.optim.AdamW,
    train_dataloaders=loaders['train'],
    val_dataloaders=loaders['val'],
    num_classes=task_config.out_features,
    max_epochs=200,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
evaluator.fit(final_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | metrics   | ModuleDict       | 0     
2 | _model    | MLP              | 5.2 K 
-----------------------------------------------
5.2 K     Trainable params
0         Non-trainable params
5.2 K     Total params
0.021     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[2024-05-14 13:33:30] Intermediate result: 0.6993932723999023  (Index 0)


...

[2024-05-14 13:41:35] Intermediate result: 0.8281555771827698  (Index 198)


Validation: |          | 0/? [00:00<?, ?it/s]

[2024-05-14 13:41:37] Intermediate result: 0.8290591835975647  (Index 199)


`Trainer.fit` stopped: `max_epochs=200` reached.


[2024-05-14 13:41:37] Final result: 0.8290591835975647


# Evaluate final model

In [12]:
test_evaluator = ClassificationEvaluator(
    learning_rate=3e-4,
    weight_decay=1e-5,
    optimizer=torch.optim.AdamW,
    val_dataloaders=loaders['test'],
    num_classes=task_config.out_features,
    max_epochs=200,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
test_evaluator.evaluate(final_model)

[2024-05-14 13:48:10] Only validation dataloaders are available. Skip to validation.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

[2024-05-14 13:48:11] Intermediate result: 0.8291265964508057  (Index 200)
[2024-05-14 13:48:11] Final result: 0.8291265964508057
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.8291265964508057
        val_loss            0.4151475727558136
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.4151475727558136, 'val_acc': 0.8291265964508057}]